In [20]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

### Attention à la gestion des points des parenthèses et des crochets
mettre *echapperParCro* à :
- *True* pour utiliser les parenthèses et les crochets dans les chaines
- *False* pour que les caractères en question soient considérés comme des éléments de **regex**

Pour l'instant, les points sont toujours échappés.

In [21]:
debug=False
echapperParCro=False

In [22]:
dossier="/Github/eGloss"
enonce="Test"
theme="LG-Separations-Morphes"
numeroterQuestions=True
adaptatifPenalite="0.2000000"
dernierIndiceNote=75.0
msgPlusDeMots=u"Il y a plus de mots"
msgMoinsDeMots=u"Il y a moins de mots"
def nbIndiceNotes(nb):
    if nb==1: 
        return ["%50%"]
    elif nb==2:
        return ["","%50%"]
    else:
        return [""]+["%"+str(int((i+2)*dernierIndiceNote/nb))+"%" for i in range(nb-1)]   

In [23]:
if debug:
    for i in range(20):
        print i,nbIndiceNotes(i) 

nbIndiceNotes={
    1:["%50%"],
    2:["","%50%"],
    3:["","%40%","%75%"],
    4:["","%25%","%50%","%75%"],
    5:["","%25%","%40%", "%60%","%75%"],
    6:["","%20%","%35%","%50%", "%60%","%75%"],
    7:["","%15","%25%","%35%","%50%", "%60%","%75%"],
    8:["","%15","%25%","%35%","%45%","%55%", "%65%","%75%"],
    9:["","%5%","%15","%25%","%35%","%45%","%55%", "%65%","%75%"],
    10:["","%8%","%16%","%24%","%32%","%40%","%48%", "%56%","%64%","%75%"],
}

In [24]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [25]:
from ToolsMoodleXML2 import *
import yaml

In [26]:
yamlFichier=base+theme+"-"+enonce+".yaml"
csvFichier=base+theme+".txt"
boolEX=True
titreNoEX=False

In [27]:
with open(yamlFichier, 'r') as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r', encoding="utf8") as file:
    donneesLignes=file.readlines()

In [28]:
def donneesEX():
    titre=[]
    result={}
    boolMultiLignes=False
    i=0
    for ligne in donneesLignes:
        if debug or 0: print ligne
        i+=1
        if ligne.startswith("#EX;"):
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#EXM;"):
            boolMultiLignes=True
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                titre=temp[1:]            
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            if boolMultiLignes:
                result["boucle"].extend(temp)
            else:
                result["boucle"].append(temp)
#            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
else:
#    print "sans #EX;"
    donneesNoEX()

In [29]:
def lierMot(ref,chaine):
    if motReferences[ref]:
        mot=motReferences[ref]
    else:
        mot=ref
    result=chaine.replace(ref,u'<a href="https://fr.wikipedia.org/wiki/%s" target="_blank" style="color:blue">%s</a>'%(mot,ref))
    return result

def echapperCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\\[\1\\]",corps)
        result=prefixe+corps+suffixe
    print "échappé",result
    return result    

def modifierCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    print "\tsubPoints", result
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()
        if echapperParCro:
            subParentheses = re.compile(ur"\(([^)]*)\)")
            corps=subParentheses.sub(ur"\\(\1\\)",corps)
            print "\tsubParentheses",corps
            subCrochets = re.compile(ur"\[([^]]*)\]")
            corps=subCrochets.sub(ur"\\[\1\\]",corps)
            print "\tsubCrochets",corps
        unites=filter(None,corps.split("&&"))
        print "\tunités",unites
        corps=".*".join(unites)
        result=prefixe+corps+suffixe
    print "\téchappé",result
    return result    


def separerMots(elements,titre):
    result=[]
    elements = filter(None, elements)
    nbMots=len(elements[0].strip().split(" "))
    solution=u"=%s#Bravo ! La phrase était : %s"%(elements[0],titre)
    corrections=[]
    nbElements=len(elements[1:])
#    if not nbElements in nbIndiceNotes: 
#        print "Ajouter une répartition des notes pour %d indices"%nbElements
#        print titre
#        print elements[1:]
    for n,element in enumerate(elements[1:]):
        (reponseSpec,feedbackSpec)=element.split("#")
        reponseSpec=modifierCorrection(reponseSpec)
        if feedbackSpec:
            for mot in motReferences:
                if mot in feedbackSpec:
                    feedbackSpec=lierMot(mot,feedbackSpec)
        element="#".join([reponseSpec,feedbackSpec])
        corrections.append(nbIndiceNotes(nbElements)[n]+element)
#    print nbMots
    minMots="^"+"([^ ]+ +)?"*(nbMots-4)+"[^ ]+$#%s"%msgPlusDeMots
    maxMots="^"+"([^ ]+ +)"*(nbMots+2)+"([^ ]+ +)*[^ ]+$#%s"%msgMoinsDeMots
#    print "~".join(elements)
    result.extend([solution,minMots,maxMots])
    result.extend(corrections)
    return result

In [30]:
consigne="<br/>".join(configuration[u"Consigne"])
motReferences=configuration[u"Références"]
separationExercices=[]
for numQuestion,question in enumerate(donnees):
    questionLignes=[]
    for nom in question:
        titre=nom
        if numeroterQuestions :
            titreQuestion="%02d-"%(numQuestion+1)+nom
        else:
            titreQuestion=nom
        questionLignes.append(consigne)
        #####
        #
        # Ajouté le égal = dans le split pour les clitiques
        #
        ####
        questionLignes.append(u"".join(re.split("[ -=]",question[nom]["boucle"][0])))
#        questionLignes.append(u"".join(question[nom]["boucle"][0].split(" ")))
        questionLignes.append(u"{1:RXC:%s}"%u"~".join(separerMots(question[nom]["boucle"],titre)))
#        print questionLignes
        textQuestion=u"<br/>".join(questionLignes)
        separationExercices.append(ClozeExercice(titreQuestion,textQuestion,penalty=adaptatifPenalite,nbHints=3))

original --.* kɥizin
	subPoints --.* kɥizin
	échappé --.* kɥizin
original --.* uvʁə-bwat .*
	subPoints --.* uvʁə-bwat .*
	échappé --.* uvʁə-bwat .*
original --.* d-?ə l-?a .*
	subPoints --.* d-?ə l-?a .*
	échappé --.* d-?ə l-?a .*
original --.* də .*
	subPoints --.* də .*
	échappé --.* də .*
original --.* œ̃-n .*
	subPoints --.* œ̃-n .*
	échappé --.* œ̃-n .*
original --.* l-a .*
	subPoints --.* l-a .*
	échappé --.* l-a .*
original --amɛn(-|=)?mwa .*
	subPoints --amɛn(-|=)?mwa .*
	échappé --amɛn(-|=)?mwa .*
original --amɛn=mwa .*
	subPoints --amɛn=mwa .*
	échappé --amɛn=mwa .*
original --.*(&& ɑ̃fɑ̃ && aʁiv-?e && lise && bys).*
	subPoints --.*(&& ɑ̃fɑ̃ && aʁiv-?e && lise && bys).*
	unités [u' \u0251\u0303f\u0251\u0303 ', u' a\u0281iv-?e ', u' lise ', u' bys']
	échappé --.*( ɑ̃fɑ̃ .* aʁiv-?e .* lise .* bys).*
original --s-?ɛt .*
	subPoints --s-?ɛt .*
	échappé --s-?ɛt .*
original --.* ɛ-?t .*
	subPoints --.* ɛ-?t .*
	échappé --.* ɛ-?t .*
original --.* aʁiv-e .*
	subPoints --.* aʁiv-e .*
	

	subPoints --.*(&&e && e && k-?i ).*
	unités [u'e ', u' e ', u' k-?i ']
	échappé --.*(e .* e .* k-?i ).*
original --.*(&& sɑ̃-?z ok-?œ̃-?n && d-?e ).*
	subPoints --.*(&& sɑ̃-?z ok-?œ̃-?n && d-?e ).*
	unités [u' s\u0251\u0303-?z ok-?\u0153\u0303-?n ', u' d-?e ']
	échappé --.*( sɑ̃-?z ok-?œ̃-?n .* d-?e ).*
original --.*(&& sɑ̃-z ok-?œ̃-n && d-e ).*
	subPoints --.*(&& sɑ̃-z ok-?œ̃-n && d-e ).*
	unités [u' s\u0251\u0303-z ok-?\u0153\u0303-n ', u' d-e ']
	échappé --.*( sɑ̃-z ok-?œ̃-n .* d-e ).*
original --.* okœ̃-n .*
	subPoints --.* okœ̃-n .*
	échappé --.* okœ̃-n .*
original --.* k-?i .*
	subPoints --.* k-?i .*
	échappé --.* k-?i .*
original --.*(&& ɔ̃(-|=)?nu-?z(-|=)?ɛ̃fliʒ && nu-?z(-|=)?afliʒ).*
	subPoints --.*(&& ɔ̃(-|=)?nu-?z(-|=)?ɛ̃fliʒ && nu-?z(-|=)?afliʒ).*
	unités [u' \u0254\u0303(-|=)?nu-?z(-|=)?\u025b\u0303fli\u0292 ', u' nu-?z(-|=)?afli\u0292']
	échappé --.*( ɔ̃(-|=)?nu-?z(-|=)?ɛ̃fliʒ .* nu-?z(-|=)?afliʒ).*
original --.*(&& ɔ̃=nu-z=ɛ̃fliʒ && nu-z=afliʒ).*
	subPoints --.*(&& ɔ̃=n

In [31]:
categorie=configuration[u"Catégorie"]
clozeQuestions=XMLSeries(categorie)

In [32]:
for exercice in separationExercices:
    clozeQuestions.addExercice(exercice)

In [33]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(clozeQuestions.getSeries())

In [34]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [35]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [36]:
motReferences

{'clitique': None,
 'datif': None,
 u'd\xe9monstratif': u'Adjectif_d\xe9monstratif',
 'liaison': u'Liaison_en_fran\xe7ais',
 'partitif': 'Article_partitif',
 'possessif': None,
 u'pr\xe9position': None,
 u'r\xe9fl\xe9chi': u'Morphologie_du_pronom_personnel_en_fran\xe7ais#Pronom_r.C3.A9fl.C3.A9chi_et_pronom_non_r.C3.A9fl.C3.A9chi'}

In [37]:
correction=u"--.*(&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]).*"
#correction=u"--.*(&& DEF-M.SG.⊖ chat&& DEF-F.SG.⊖ gouttière).*"
(prefixe, corps, suffixe)=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",correction).groups()
print corps
subParentheses = re.compile(ur"\(([^)]*)\)")
result=subParentheses.sub(ur"\\(\1\\)",corps)
print result
subCrochets = re.compile(ur"\[([^]]*)\]")
result=subCrochets.sub(ur"\\[\1\\]",result)
subPoints = re.compile(ur"(\.[^*])")
result=subPoints.sub(ur"\\\1",result)

print prefixe+result+suffixe

&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]
&& gris\(M\)[SG.⊖] && vert\(M\)[SG.⊖] && clair[M.SG.⊖]
--.*(&& gris\(M\)\[SG\.⊖\] && vert\(M\)\[SG\.⊖\] && clair\[M\.SG\.⊖\]).*


In [38]:
subParentheses = re.compile(ur"&&([^&)]*)\(([^)]*)\)")
subCrochets = re.compile(ur"&&([^&[]*)\[([^\]]*)\]")
subPoints = re.compile(ur"&&(([^)\.]|\\\)|\\\.)+)(\.[^*])")
print subPoints.search(correction)
result=subParentheses.sub(ur"&&\1\\(\2\\)",correction)
print result
result=subCrochets.sub(ur"&&\1\\[\2\\]",result)
print result
result=subPoints.sub(ur"&&\1\\\3",result)
print result

--.*(&& gris\(M\)[SG.⊖] && vert\(M\)[SG.⊖] && clair[M.SG.⊖]).*
--.*(&& gris\(M\)\[SG.⊖\] && vert\(M\)\[SG.⊖\] && clair\[M.SG.⊖\]).*
--.*(&& gris\(M\)\[SG\.⊖\] && vert\(M\)\[SG\.⊖\] && clair\[M\.SG.⊖\]).*
